In [2]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = (boston_housing.load_data())

print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


## preparing the data

In [3]:
#we normalize the data before it's fed to the model
#import numpy as np
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

#print(mean)

In [4]:
#building the model
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss='mse', metrics=["mae"])
    return model




## k-fold validation

In [5]:
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
     train_data[(i+1) * num_val_samples:]],
    axis=0)
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
     train_targets[(i+1) * num_val_samples:]],
    axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=16, verbose=0)
    #trains the model int silent mode verbose=0
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)#evaluates the model on the validation data
    all_scores.append(val_mae)

Processing fold #0
Processing fold #1
Processing fold #2
Processing fold #3


In [6]:
print(all_scores)

[1.901277780532837, 2.5186820030212402, 2.422339916229248, 2.3837592601776123]


In [7]:
print(np.mean(all_scores))

2.3065147399902344
